# Item 37: Compose Classes Instead of Nesting Many Levels of Built-in Types

Python's dictionary type is very useful for when maintaining dynamic internal state over the lifetime of an object.

In [1]:
# Say we want to record the grades of a set of students whose names aren't known in advance. We can define a 
# class to store the names in a dictionary instead of using a predefined attribute for each student
class SimpleGradebook:
    def __init__(self):
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = []

    def report_grade(self, name, score):
        self._grades[name].append(score)

    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [2]:
# Using the class is simple
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
book.report_grade('Isaac Newton', 95)
book.report_grade('Isaac Newton', 85)

print(book.average_grade('Isaac Newton'))

90.0


Dictionaries and their related built-in types are so easy to use that there's a danger of overextending them to write brittle code.

Say, for example, we wanto to extend the `SimpleGradebook` class to keep a list of grades by subject, not just overall. We can do this by changing the `_grades` dictionary to map student names (its keys) to yet another dictionary (its values). The innermost dictionary will map subjects (its keys) to a `list` of grades (its values). 

In [5]:
# Here, we do this by using a defaultdict instance for the inner dictionary to handle missing subjects
from collections import defaultdict

class BySubjectGradebook():
    def __init__(self):
        self._grades = {} # Outer dict

    def add_student(self, name):
        self._grades[name] = defaultdict(list) # Inner dict
    
    # Because of this change, the report_grade and average_grade 
    # methods gain a bit of complexity, but they are till manageable

    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(grade)

    def average_grade(self, name):
        by_suject = self._grades[name]
        total, count = 0, 0
        for grades in by_suject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

In [6]:
# Using the above class remains simple enough
book = BySubjectGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75)
book.report_grade('Albert Einstein', 'Math', 65)
book.report_grade('Albert Einstein', 'Gym', 90)
book.report_grade('Albert Einstein', 'Gym', 95)
print(book.average_grade('Albert Einstein'))

81.25


In [ ]:
# Now lets say that we want to track the weight of each score toward the overall grade in the class so that
# midterms and final exams are more important than quizes. We can implement this by using a tuple in the 
# values list
class WeightedGradeBook():
    def __init__(self):
        self._grades = {} # Outer dict

    def add_student(self, name):
        self._grades[name] = defaultdict(list) # Inner dict

    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))

    # Although the changes to the report_grade method were simple, the averag_grade method
    # now has a loop within a loop making it difficult to read

    def average_grade(self, name):
        by_suject = self._grades[name]

        score_sum, score_count = 0, 0
        for subject, scores in by_suject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            
            score_num